In [17]:
import sys
import numpy as np

import torch.nn as nn
import torch

import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

sys.path.append('../')
import utils.DataGen as DataGen
from utils.Model import Model

In [18]:
xmin = -8#param {type:"number"}
xmax = 8#param {type:"number"}
N=15#param {type:"slider", min:5, max:30, step:1}
n_state=0#param {type:"slider", min:0, max:10, step:1}
n_points = 100#param {type:"integer"}
n_samples=2000#param {type:"integer"}
#markdown ---

# We generate the values of alpha
alpha_min = np.array([-150,-10,0.3,-0.1,0])/50
alpha_max = np.array([50,5,1.0,0.1,0.25])/50

k = alpha_min.shape[0]

r_alpha = np.random.random((int(n_samples*0.8), k)) # Values between 0 and 1
alpha1 = r_alpha*(alpha_max - alpha_min)+ alpha_min # random alpha

alpha_min2 = np.array([-10,-1,0.1,0,0])
alpha_max2 = np.array([5,1,1.0,0,0])

r_alpha = np.random.random((int(n_samples*0.2), k)) # Values between 0 and 1
alpha2 = r_alpha*(alpha_max2 - alpha_min2)+ alpha_min2 # random alpha
alpha = np.concatenate((alpha1, alpha2))



In [ ]:
data_gen = DataGen.potencial_datos(alpha_min, alpha_max, N)

# Generate the energies, wavefunctions and potentials
E, a, alpha = data_gen.generate_data(n_samples, alpha)
waves, x,phis = data_gen.final_wavefunction( xmin, xmax, n_points, a)
V, _ = data_gen.evaluar_potencial( xmin, xmax, n_points, alpha)
idx=-1

In [20]:
V = np.loadtxt('../in/V.txt', dtype=float)
waves = np.loadtxt('../in/waves.txt', dtype=float)
print(f'V.shape = {V.shape}')
print(f'waves.shape = {waves.shape}')

V.shape = (2000, 100)
waves.shape = (2000, 100)


In [21]:
from sklearn.model_selection import train_test_split

# Split train and test 
idx_train, idx_test, wave_train, wave_test = train_test_split(np.arange(V.shape[0]),
                                                              waves, test_size=0.3, random_state=123)
V_train = V[idx_train,:]
V_test = V[idx_test,:]
print(f'V_train.shape = {V_train.shape}')
print(f'V_test.shape = {V_test.shape}')

V_train.shape = (1400, 100)
V_test.shape = (600, 100)


In [22]:
model = Model(n_points, n_points)

In [23]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [24]:

V_train = torch.from_numpy(V_train).float()
wave_train = torch.from_numpy(wave_train).float()

epochs = 100
batch_size = 100
log_each = 1
l = []
model.train()
batches = len(V_train) // batch_size
for e in range(1, epochs + 1):
    _l = []
    for b in range(batches):
        x_b = V_train[b*batch_size:(b+1)*batch_size]
        y_b = wave_train[b*batch_size:(b+1)*batch_size]

        y_pred = model(x_b)
        loss = criterion(y_pred, y_b)

        _l.append(loss.item())

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    l.append(np.mean(_l))
    if not e % log_each:
        print(f'Epoch {e}/{epochs} Loss {np.mean(l):.5f}')

Epoch 1/100 Loss 0.05052
Epoch 2/100 Loss 0.03878
Epoch 3/100 Loss 0.03216
Epoch 4/100 Loss 0.02776
Epoch 5/100 Loss 0.02437
Epoch 6/100 Loss 0.02167
Epoch 7/100 Loss 0.01952
Epoch 8/100 Loss 0.01778
Epoch 9/100 Loss 0.01633
Epoch 10/100 Loss 0.01509
Epoch 11/100 Loss 0.01404
Epoch 12/100 Loss 0.01312
Epoch 13/100 Loss 0.01233
Epoch 14/100 Loss 0.01163
Epoch 15/100 Loss 0.01100
Epoch 16/100 Loss 0.01045
Epoch 17/100 Loss 0.00995
Epoch 18/100 Loss 0.00951
Epoch 19/100 Loss 0.00910
Epoch 20/100 Loss 0.00872
Epoch 21/100 Loss 0.00838
Epoch 22/100 Loss 0.00806
Epoch 23/100 Loss 0.00777
Epoch 24/100 Loss 0.00751
Epoch 25/100 Loss 0.00726
Epoch 26/100 Loss 0.00703
Epoch 27/100 Loss 0.00682
Epoch 28/100 Loss 0.00662
Epoch 29/100 Loss 0.00643
Epoch 30/100 Loss 0.00626
Epoch 31/100 Loss 0.00609
Epoch 32/100 Loss 0.00594
Epoch 33/100 Loss 0.00579
Epoch 34/100 Loss 0.00565
Epoch 35/100 Loss 0.00552
Epoch 36/100 Loss 0.00539
Epoch 37/100 Loss 0.00527
Epoch 38/100 Loss 0.00516
Epoch 39/100 Loss 0.0

In [25]:
model.eval()

Model(
  (c1): Linear(in_features=100, out_features=256, bias=True)
  (c2): ReLU()
  (c3): Dropout(p=0.1, inplace=False)
  (c4): Linear(in_features=256, out_features=256, bias=True)
  (c5): ReLU()
  (c6): Dropout(p=0.1, inplace=False)
  (c7): Linear(in_features=256, out_features=128, bias=True)
  (c8): ReLU()
  (c9): Dropout(p=0.1, inplace=False)
  (c10): Linear(in_features=128, out_features=128, bias=True)
  (c11): ReLU()
  (c12): Dropout(p=0.1, inplace=False)
  (c13): Linear(in_features=128, out_features=100, bias=True)
)

In [26]:
V_test.dtype

dtype('float64')

In [34]:
V_test = torch.from_numpy(V_test).float()
wave_test = torch.from_numpy(wave_test).float()

predict = model(V_test)

print("Test MSE: %f" % criterion(predict, wave_test))

Test MSE: 0.000101


In [35]:
predict = predict.detach().numpy()

xmin = -8
xmax = 8
x = np.arange(xmin, xmax, (xmax - xmin)/n_points)

In [38]:
idx = 3
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=predict[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=wave_test[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=V_test[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Random potential prediction",
    xaxis_title="x")

fig.show()